<a href="https://colab.research.google.com/github/lu-lab/frcnn-all-in-one/blob/main/Faster_R_CNN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Faster R-CNN training
---
This notebook will allow you to train a custom object detector using Google's GPU resources. 
If you're here and have never used a Jupyter notebook before, it's a straightforward way to interact with code and see the visual outputs of code snippets. There are two types of snippets, or 'cells', text cells, like this one, and code cells, like the cell that follows this. You can run individual code cells by clicking the 'play' button to the left of code cells.

Enable the GPU by going to Runtime -> Change runtime type and select "GPU" from the dropdown menu. This will speed your training time up substantially, but note that Google has a limit on how much of this GPU resource you can use. If you use the GPU resource heavily, you may have to subscribe to a paid plan. In this notebook, we will be annotating images from your custom dataset and then fine-tuning the Faster-RCNN network (specifically, the Faster R-CNN Inception ResNet V2) starting from a model pre-trained on the COCO 2017 image set. This pre-trained model is provided in the Tensorflow 2 Model Zoo [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). 

We recommend reading this notebook through **in full** before starting. Then, you can run the cells in this notebook **in order**, being certain not to skip any unless they are marked as 'Optional'. 


###Step 0: Copy this notebook


---


First off, **save a copy of this notebook to your own Google Drive!** This is partially to protect your data and partially so you can save any changes you may want to make to this notebook. We recommend putting it into it's own folder, because we will be making quite a few sub-folders to organize the inputs and outputs of the network and the code itself.

**Note**: Following this notebook will require several GB of space in your Google Drive, in addition to whatever space you may need for your annotated image data, or any data you may want to perform inferences on. 



###Step 1: Connect to your Google Drive

---


The first code cell below will mount Google Drive, get files we need from the GitHub repository to run this notebook, and make a few new folders that we will put data in. 

Before running the cell below, make sure to modify the path following the first '%cd' to the path this notebook is in! Any filepath within Google Drive starts with '/content/drive/'. To check where this notebook exists, go to File -> Locate in Drive. A new browser tab will open. Navigate to the new tab and below the 'Search' box you will see the rest of the path. For example, if you see 'My Drive > Colab Notebooks', the full path would be
```
/content/drive/My Drive/Colab Notebooks/
```
If you have spaces in the path, add a \ before the space. For example, the path 

```
/content/drive/My Drive/Colab Notebooks/Faster R-CNN/
```

becomes 
```
/content/drive/My\ Drive/Colab\ Notebooks/Faster\ R-CNN/
```

Now, make sure your Runtime type is set to GPU (Runtime -> Change runtime type and select "GPU" from the dropdown menu), and run the following cell. 

Another note on using this notebook: if you need to step away, and your computer goes inactive, Colab will eventually stop your session. That means you'l need to restart the session (Hit reconnect in the upper right bar). If you can, step away at the end of a step, because that way your progress through the code (for example, your annotations and tfrecord creation) will be saved to your Google Drive. **On the first line of every code cell** you will see a note about whether you *have* to run that cell every time you reconnect.

In [ ]:
# Run this cell every time you run this notebook (every time you reconnect)

# First we need to mount Google drive and gather some dependencies...
from google.colab import drive 
drive.mount('/content/drive')
import os

# NOTE: Modify this path if needed
%cd /content/drive/My\ Drive/Colab\ Notebooks/Faster\ R-CNN/
working_dir = os.getcwd()

In [ ]:
# Run this cell only the first time you run this notebook 
!pip install --upgrade pip

!git clone https://github.com/lu-lab/frcnn-all-in-one.git
%cd {working_dir}
!cp -a ./frcnn-all-in-one/colab/. .
!rm -r ./frcnn-all-in-one
!rm Faster_R_CNN_training.ipynb

# if any of these folders already exist, they will not be made, but that is not a problem
!mkdir annotations
!mkdir images
!mkdir exported-model
!mkdir inferencing-results


###Step 2: Install libraries

---

The next cell will install all the necessary Python libraries and packages to train our model to this notebook. At the end of this step, you'll also have several new folders, including Tensorflow, COCO-trained-model, and cocoapi. The Tensorflow directory includes most of the model development pipeline, the cocoapi allows the pipeline to compute metrics describing how well the model performs as we train it, and the COCO-trained-model is our starting model that we will be fine-tuning.

If you see errors in the output to this step, note that it doesn't necessarily mean the rest of the notebook won't work. For example you might see the following errors, which you can ignore:
```
ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.30.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.
```

**Note**: If you've already run this notebook once, you do NOT need to re-download the Tensorflow, cocoapi, and COCO-trained-model files, but you DO need to install them and other libraries again if you have stopped the session! If you just need to re-install everything, put a # in front of every line in the code below that has a # at the end of it.

In [ ]:
# Run this cell every time you run this notebook (every time you reconnect)
%mkdir Tensorflow 
%cd ./Tensorflow
!git clone https://github.com/tensorflow/models.git #
%cd {working_dir}
!apt-get -qq install -y protobuf-compiler python-pil python-lxml python-tk 
%cd ./Tensorflow/models/research/
!protoc object_detection/protos/*.proto --python_out=.
%cd {working_dir}

!git clone https://github.com/cocodataset/cocoapi.git #
%cd ./cocoapi/PythonAPI
!make
coco_tools_path = os.path.join(working_dir, "Tensorflow/models/research")
!cp -r pycocotools {"%r"%coco_tools_path}
%cd {working_dir}

%cd Tensorflow/models/research/
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd {working_dir}

%mkdir COCO-trained-model
%cd COCO-trained-model
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz #
!tar -xf faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz #
!rm faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz #
coco_model_path = os.path.join(working_dir, "COCO-trained-model")
!mv -v faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/* {"%r"%coco_model_path} #
!rmdir faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8 #
%cd {working_dir}

!cp ./Tensorflow/models/research/object_detection/model_main_tf2.py . #
!cp ./Tensorflow/models/research/object_detection/exporter_main_v2.py . #

!pip install opencv-python-headless==4.4.0.44

%load_ext tensorboard

In [ ]:
#Run this cell if you have not completed step 3 & 4 in a previous session for your data

# import all the packages we'll need for the rest of the notebook and make a few 
# helper functions that we'll use to build our own dataset
import glob 
from math import floor
from itertools import chain
import random

import tensorflow as tf
from six import BytesIO
import cv2
from PIL import Image
import pandas as pd
import numpy as np

from object_detection.utils import colab_utils
from object_detection.utils import visualization_utils as viz_utils

max_width = 600

def load_image_into_numpy_array(path):
  image = cv2.imread(path)
  (im_width, im_height, im_depth) = image.shape
  ratio = max_width/im_width
  im_width = floor(im_width*ratio)
  im_height = floor(im_height*ratio)
  image = cv2.resize(image, (im_height, im_width), interpolation = cv2.INTER_LANCZOS4)
  return image

def write_label_map(classes, label_map_path):
    s = ' '
    for ID, name in enumerate(classes):
        out = ''
        out += 'item' + s + '{\n'
        out += '\t' + 'id:' + s + (str(ID+1)) + '\n'
        out += '\t' + 'name:' + s + '\'' + name + '\'' + '\n'
        out += '}\n\n'
        with open(label_map_path, 'a') as f:
            f.write(out)

def write_tf_record(annotations, tfrecord_path, img_path, label_map):
    tf_writer = tf.io.TFRecordWriter(tfrecord_path)
    for annotation in annotations:
        tf_example = gt.create_tf_example(annotation, img_path, label_map)
        tf_writer.write(tf_example.SerializeToString())

    tf_writer.close()
    output_path = os.path.join(os.getcwd(), tfrecord_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

###**Step 3: Annotate your data**


---


Before you can train your model, you need a **set of annotations** for your images. First, the images that you would like to annotate need to be uploaded to the folder called 'images' in the same Google Drive directory as this notebook. The images folder has already been created for you in a previous step, so find the images you'd like to annotate and upload them now!

If you've completed Step 3 in a previous session you can skip ahead to Step 4.

#### **How to choose what images to annotate and how many**

---


Ideally, you should annotate images that encompass the imaging conditions you expect to encounter in your dataset. This will allow you to train a network on data representative of your dataset. Often you don't actually need to annotate that many images - in many cases as few as 10 or 20 annotated images will do just fine! If you think your dataset may be particularly tricky, you may wish to annotate on the order of ~100s of images.
 
 ---


Once you've uploaded the images you'd like to annotate, you will need to run the following three cells in order **for each type of object (class) you would like to annotate**. Draw boxes around the types of objects you wish to annotate ('classes'). For example, if you're annotating worms and eggs (2 classes), this would be what you would do:
  

1.   Use the form to the right of the cell below this to type in worm as the current_class and run the cell. 
2.   Then you'll run the cell after that and see one of your images pop up and you can annotate each worm in each image. **After annotating all your images, make sure to hit the submit button that before continuing to the next cell.**
3.   Run the cell that follows where your images popped up. This will create a table that has all the information from your annotations.
4. **Return to the cell below this one**, type in egg as the current_class, and run the cell.
5. Run the cell after that - you'll see the same images pop up again, but this time you'll be annotating eggs. **Make sure to hit the submit button before moving to the next cell.**
6. Run the cell that follows where your images popped up. This time, your new annotations will be added to the same table as your old annotations.

If you have just one class, you would simply run every cell in this section once. If you have more than two classes, continue to repeat running the following three cells until you've annotated all your classes. 

In [ ]:
#@title What's the class you're about to annotate?

current_class = 'bacteria' #@param {type:"string"}

In [ ]:
# Run this cell if you have never completed Step 3 for your data

img_path = './images'
image_files = glob.glob('./images/*.jpg') # if you have another type of image file, exchange '.jpg' out with whatever image file extension your data has. 
image_files.sort(key=os.path.getmtime)
images_np = [load_image_into_numpy_array(str(p)) for p in image_files]
boxes = []
colab_utils.annotate(images_np, box_storage_pointer=boxes)

In [ ]:
# Run this cell if you have never completed Step 3 for your data

from os.path import exists

# populate filenames
annotation_df = pd.DataFrame() 
filenames_for_boxes = [[name]*len(boxes[i]) for i, name in enumerate(image_files)]

# Grab widths and heights for each image 
widths = np.zeros((len(image_files),), dtype='int')
heights = np.zeros((len(image_files),), dtype='int')
for i, im_file in enumerate(image_files):
    im = cv2.imread(im_file)
    heights[i], widths[i], d = im.shape

annotation_df['filename'] = list(chain.from_iterable(filenames_for_boxes))

# format of boxes matrix returned from annotation is [y_min, x_min, y_max, x_max]
annotation_df['ymin']  = [int(box[0]*heights[i]) for i, img in enumerate(boxes) for box in img]
annotation_df['xmin'] = [int(box[1]*widths[i]) for i, img in enumerate(boxes) for box in img]
annotation_df['ymax'] = [int(box[2]*heights[i]) for i, img in enumerate(boxes) for box in img]
annotation_df['xmax'] = [int(box[3]*widths[i]) for i, img in enumerate(boxes) for box in img]

annotation_df['width'] = annotation_df['xmax'] - annotation_df['xmin']
annotation_df['height'] = annotation_df['ymax'] - annotation_df['ymin']
annotation_df['class'] = [current_class]*len(annotation_df)

# screen out any un-annotated frames (boxes will be filled with zeros when un-annotated)
annotation_df = annotation_df[(annotation_df['xmin'] != annotation_df['xmax'])]


# save csv to annotations directory - if the file exists, we will append to it so that you can annotate multiple classes. 
csv_filepath = './annotations/bounding_boxes.csv'
if os.path.isfile(csv_filepath):
  annotation_df.to_csv(csv_filepath, index=False, mode='a', header=False)
else:
  annotation_df.to_csv(csv_filepath, index=False)



After completing your annotation of **all classes** in your image set, run the following cell to make a text file that maps your class names to numbers and to split your annotations into images to train the neural network on and images to test the neural network on. 

In [ ]:
# Run this cell if you have never completed Step 3 for your data

# if you want to change the train/ test split, you can decrease number of training images (which will increase the testing images) by decreasing this. It must be between 0 and 1
train_split = 0.9 

# load all the annotations from the csv file
csv_filepath = './annotations/bounding_boxes.csv'
annotation_df = pd.read_csv(csv_filepath)

# write a label map so we can map a number to each annotated class
label_map_path = './training/label_map.pbtxt'
write_label_map(annotation_df['class'].unique().tolist(), label_map_path)

# randomly choose images to be train or test images
annotation_df['filename'] = annotation_df['filename'].str.replace("./images/","", regex=True)
annotated_images = annotation_df['filename'].unique()
num_train_images = int(train_split*len(annotated_images))
train_images = random.sample(list(annotated_images), num_train_images)
test_images = list(set(annotated_images).difference(set(train_images)))
test_idx = annotation_df['filename'].isin(test_images)
test_or_train = ['test' if test_i else 'train' for test_i in test_idx]
annotation_df['test_or_train'] = test_or_train

#overwrite the old file so that it includes the test and train sets. 
annotation_df.to_csv(csv_filepath, index=False)

### **Step 4: Generate tfrecord files**

---


Now we want to use the bounding_boxes.csv and label_map.pbtxt file to generate tfrecords. Tfrecords are the format that tensorflow uses to input training data and to test the network on test data as the model trains. Put the 'bounding_boxes.csv' file in the 'annotations' folder and the 'label_map.pbtxt' file in the 'training' folder before running the following cell.








In [ ]:
# Run this cell if you have not previously completed Step 4 for your data

import sys
sys.path.append("./Tensorflow/models/research/object_detection")
from object_detection.utils import label_map_util
import generate_tfrecord as gt

label_map_path = './training/label_map.pbtxt'
csv_filepath = './annotations/bounding_boxes.csv'

# and now we'll convert annotations to a tfrecord
label_map = label_map_util.get_label_map_dict(label_map_path)
all_annotations = pd.read_csv(csv_filepath)
img_path = './images'

# tfrecord for train annotations
tfrecord_train_path = './annotations/train.record'
train_annotations = all_annotations[all_annotations['test_or_train'].isin(['train'])]
grouped_train_annotations = gt.split(train_annotations, 'filename')
write_tf_record(grouped_train_annotations, tfrecord_train_path, img_path, label_map)

# tfrecord for test annotations
tfrecord_test_path = './annotations/test.record'
test_annotations = all_annotations[all_annotations['test_or_train'].isin(['test'])]
grouped_test_annotations = gt.split(test_annotations, 'filename')
write_tf_record(grouped_test_annotations, tfrecord_test_path, img_path, label_map)

### **Step 5: Train the model**

---



Now we will fine-tune the model with our own images and classes. Note that if anything is not in it's proper folder when you run this step, it will fail. You will see a lot of warnings as this step starts to run - not to worry, as long as you don't see an error it should be ok! 

This step will take a long time, even with the GPU. It is up to you when to stop training (you can go to Runtime ->Interrupt Execution), and generally a good rule of thumb is to watch for when the total loss starts to plateau (this may take an hour or so). We recommend starting with about an hour of training, assessing the model, then if needed you can train the model more. You should be able to monitor the loss by watching the Tensorboard widget that will start when you run the following cell. Once the first 100 training steps are complete, click the refresh icon in the upper right corner of the Tensorboard widget and you will see the board populate. You will also see a lot of outputs above the Tensorboard widget that will report the total loss. The beginning of this step is slow (on the order of ten minutes), so be patient if you're not immediately seeing outputs!

To stop the training, hit the 'stop' button in the upper left of the code cell, or put your cursor in the box that is above the Tensorboard widget and hit Ctrl+C. The default here is to save the model's state every 1000 steps, which is about every half hour.

In [ ]:
# Run this cell if you have not previously completed Step 5 for your data

%load_ext tensorboard
import tensorflow as tf
import datetime

%tensorboard --logdir training/train/
!python model_main_tf2.py --model_dir=training --pipeline_config_path=./training/faster_rcnn.config --alsologtostderr

### **Step 6: Export model**

---

Once the loss becomes reasonable and you've stopped training, freeze the model and save it to the 'exported-model' folder. You'll see quite a few warnings when you run the following cell, but again no need for concern unless you see an error. Once this is done, double check your 'exported-model' folder. It should now contain a 'checkpoint' and 'saved-model' folder and a 'pipeline.config' file.

In [ ]:
# Run this cell if you have not previously completed Step 6 for your current data and model
!python exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path ./training/faster_rcnn.config \
--trained_checkpoint_dir ./training/ \
--output_directory ./exported-model

### **Step 7: Test inferencing (Optional)**

---



Now our model is exported, we can use it to inference, or predict the bounding boxes for the classes we trained on with new images that we didn't annotate. Below are a few different options for how to do this, depending on whether the images you want inference for is a list of images, a folder of images, or a movie. This step will help you visualize detections and give you a qualitative idea of how well your model performs. You can run any combination of the cells below, and you can run each cell multiple times if you want. But, take care to rename the output files each time you run the same cell multiple times, or you may overwrite data from an earlier run! 

Here are the filetypes you should expect from each of the following three cells ("Inferencing for x"). These filetypes will be saved to the 'inferencing-results' folder:


*   h5 (hdf) file with bounding box inferences for each image or frame in the list, folder, or movie. You can convert this to a .csv using the code cell that follows the three inferencing examples. 
*   images (.jpg format) or a movie (.mp4 format). At the end of each inferencing cell, the function 'inferencing_tools.label_all_detections_from_h5' takes as input the path of the h5 file and the paths to the original image data and uses it to overlay boxes where the model has  detected objects on the original images. If the image input is a list of image paths or a folder, the output will be in image format, and if the image input is a movie, the output will be in movie format. If you do not wish to do this for every set of images or movie you process, you can comment out the line containing 'inferencing_tools.label_all_detections_from_h5' in each of the following three cells. 

There are two other variables below that you may be interested in changing:


*   ```
target_classes
```
*    ```
target_min_scores
```

```target_classes``` is a list of the classes that you would like the detector to pay attention to. This is based off of the label_map.pbtxt file that contains the names of the objects you annotated and the number that they're identified with during training. If you open the label_map.pbtxt file (which you can do from the sidebar of this notebook once your google drive is connected - click the file icon -> drive -> MyDrive -> wherever this file is ->training ->label_map.pbtxt), you will see that each class has a corresponding id. The ```target_classes``` list is index shifted, so whatever has an id of '1' in your label_map.pbtxt file must be represented in your ```target_classes``` by the number 0. If you followed our annotation notebook exactly, '0' should represent a worm and '1' should represent an egg because in the label_map.pbtxt file the worm has an id of '1' and the egg has an id of '2'. 

 ```target_min_scores``` is used to screen detections. Whenever the model identifies an object, it also provides a score describing the confidence of the identification. A value closer to 1 is high confidence, while a value closer to 0 is low confidence. For each class, you can provide a score threshold so that you can ignore low confidence predictions. Here, for worms (or whatever corresponds to the first class in the ```target_classes``` list), we use a threshold of 0.8, and for eggs (or whatever corresponds to the second class in the ```target_classes``` list) we use a threshold of 0.3.

Note that the length of the ```target_classes``` and ```target_min_scores``` lists must be equal.

For convenience, we also provide a converter to save h5 data to .csv files that can be read by Excel or Google Sheets. We recommend doing this so you have an easily human-readable copy of your data. To convert your data, you can run the cell 'Convert h5 file to csv' that follows the three different inferencing cells.


#### **Inferencing for our test images**
First, let's see how the model performs on our annotated test images. Note that this will not inference on every image you annotated, just the ones we set aside as 'test' images. We set these images aside because it wouldn't be 'fair' if we tested out the model on the exact same images we trained it on - it wouldn't give us a realistic idea of how well the model will perform on frames from a new set of images. 

You do not need to alter anything in the code below to check the detections in your test images. The following code will gather all the detections for each test image and superimpose boxes indicating where the objects are that correspond to the classes in 'target_classes'. The images with boxes imposed will be displayed in the output of the following cell, and they'll be saved in the 'inferencing-results' folder, with each image having the same name as the original test image. 

In [ ]:
import sys
import glob
import IPython
from IPython.display import Image, display
import cv2
import pandas as pd
from inference_code import inferencing_tools

# Set up all the tensorflow files we'll need as inputs
path_to_config = './exported-model/pipeline.config'
path_to_ckpt = './exported-model/checkpoint'
# we've already defined this above, but just in case
path_to_labels = './training/label_map.pbtxt'

# Add hdf file info
save_to_hdf = True
save_path = './inferencing-results'
# NOTE: change the h5_file variable  so you use a different filename each time 
# you run this cell ! For example, change 'test_detections.h5' in the code below 
#to 'test_detections_2.h5' This will make sure you don't overwrite any data
h5_file = os.path.join(save_path, 'test_detections.h5')

# pull out the training image names from the bounding_boxes.csv file
csv_filepath = './annotations/bounding_boxes.csv'
all_annotations = pd.read_csv(csv_filepath)
test_image_names = all_annotations['filename'][all_annotations['test_or_train'].isin(['test'])].unique()

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN_tf2(path_to_ckpt, path_to_labels, save_to_hdf, h5_file, path_to_config)

# See note above on what these variables mean and how you can change them
target_classes = [0, 1]
# Confidence score to use as a minimum threshold. Ranges from 0 to 1.
target_min_scores = [0.8, 0.1]

for fn in test_image_names:
    image_np = cv2.imread(os.path.join('./images', fn))
    #inferencing happens in this call - the h5 file will store information using the 'path' variable as part of the key
    boxes = cnn.get_detections(image_np, fn, target_classes, target_min_scores)
    print('Processing image %s' % fn)

test_image_paths = [os.path.join('./images', im_name) for im_name in test_image_names]
# Now that we have all the detections, label them on the test data and visualize the detections on each test image.
inferencing_tools.label_all_detections_from_h5(h5_file, test_image_paths, save_path, target_classes)

# The following two lines will display your test images with boxes superimposed
# in this notebook. 
for image_name in glob.glob('./inferencing-results/*.jpg'): 
    display(Image(filename=image_name))

#### **Inferencing from a folder of images**
This is very similar to the example above. The difference is that here we detect all objects in classes 1 and 2 from .png or .jpg images in a folder ```image_dir```. If you followed our annotation notebook exactly, these classes would be eggs and worms.  Edit the ```image_dir``` variable below to lead to the folder with your data! The data must be on Google Drive.

Here we again save the boxes and scores to an h5 file in the 'inferencing-results' directory. We then overlay the detections on top of the original images and save them in the 'inferencing-results' folder with the same name as the original image. 

In [ ]:
import sys
import glob
import IPython
from IPython.display import Image, display
import cv2
import pandas as pd
from inference_code import inferencing_tools

# set up inputs to the inferencer
path_to_config = './exported-model/pipeline.config'
path_to_ckpt = './exported-model/checkpoint'
path_to_labels = './training/label_map.pbtxt'
save_to_hdf = True
save_path = './inferencing-results'
# Note! each time you run this cell, you should change the 'folder_detections.h5' part of the filename
# otherwise you may overwrite data! The name can be anything you want, e.g. 'folder_detections_2.h5'
h5_file = os.path.join(save_path, 'folder_detections.h5')

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN_tf2(path_to_ckpt, path_to_labels, save_to_hdf, h5_file, path_to_config)

# take a look at qualitatively how well the model performs on images in the 'image_dir' folder
# edit the 'image_dir' variable below to reflect the where your folder with data is!
image_dir = './images'
ext_list = ['.jpg', '.png'] # if you have another file format, you can add it here. most formats should work.
image_names = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))
                             and f.endswith(tuple(ext_list))]

# See note above on what these variables mean and how you can change them
target_classes = [0,1]
# Confidence score to use as a minimum threshold. Ranges from 0 to 1.
target_min_scores = [0.8, 0.3]

for fn in image_names:
    image_np = cv2.imread(os.path.join(image_dir, fn))
    #inferencing happens in this call - you can directly use the boxes this returns if you wish. They are also saved in the h5 file
    boxes = cnn.get_detections(image_np, fn, target_classes, target_min_scores)
    print('Processing image %s' % fn)

# Now that we have all the detections, label them on the test data and visualize the detections on each test image.
image_paths = [os.path.join(image_dir, fn) for fn in image_names]
inferencing_tools.label_all_detections_from_h5(h5_file, image_paths, save_path, target_classes)

# superimpose boxes on images
for image_name in glob.glob('./inferencing-results/*.jpg'): 
    display(Image(filename=image_name))

#### **Inferencing from a movie**
Here's an example that detects both eggs and worms for each frame in a video with the path assigned to ```movie_path``` and saves the detections to an h5 file.  The movie with detections overlaid can then be created with the filename assigned to the ```save_file``` variable. The ```movie_path``` must lead to a movie uploaded to Google Drive.



In [ ]:
import sys
import glob
import cv2
import pandas as pd
from inference_code import inferencing_tools

# detect and visualize detections from movie
# change this variable to wherever your movie is stored
movie_path = 'your-video-path.mp4'
save_file = './inferencing-results/test_nn.mp4'

path_to_config = './exported-model/pipeline.config'
path_to_ckpt = './exported-model/checkpoint'
# we've already defined this above, but just in case
path_to_labels = './training/label_map.pbtxt'
save_to_hdf = True
save_path = './inferencing-results'
# Note! each time you run this cell, you should change the 'movie_detections.h5' part of the filename
# otherwise you may overwrite data! The name can be anything you want, e.g. 'movie_detections_2.h5'
h5_file = os.path.join(save_path, 'movie_detections.h5')

# every inference you make with this 'cnn' object will be saved in the same h5 file
cnn = inferencing_tools.CNN_tf2(path_to_ckpt, path_to_labels, save_to_hdf, h5_file, path_to_config)

# See the note above on these variables.
target_classes = [0, 1]
# Confidence score to use as a minimum threshold. Ranges from 0 to 1.
target_min_scores = [0.8, 0.3]

vid = cv2.VideoCapture(movie_path)
idx = 1
while vid.isOpened():
    ret, image = vid.read()
    if ret:
        #inferencing happens in this call
        boxes = cnn.get_detections(image, idx, target_classes, target_min_scores)
    else:
        break
    print("Processing frame no %s" % idx)
    idx += 1
vid.release()

# if the input of the inferencing is a video, the output will be a video
inferencing_tools.label_all_detections_from_h5(h5_file, movie_path, save_file, target_classes)

#### **Convert h5 file to csv**

Before running this cell, you **must** run one of the inferencing cells above, or you'll get an error.

By replacing the path assigned to ```h5_file```
below with the path to your own h5 file, you can convert it to a csv file. The current set up would allow you to convert the file ```'./inferencing-results/folder_detections.h5'``` to a csv file ```'./inferencing-results/bounding_boxes_from_folder.csv'```. If you used the inferencing cell above that performs inferencing on a folder of images with the default naming, the cell below will work as is, otherwise, simply modify the ```h5_file``` variable. The default location of the ```csv_filepath``` will save to the 'inferencing-results' folder, but feel free to rename or relocate this file as you like. 


In the resulting file, the 'frame' column is either the movie frame number or image name, and the 'xmin', 'xmax', 'ymin', and 'ymax' columns are all expressed as a proportion of the total size of the image or frame (e.g. the upper left corner of a box should be ($x_{min}*width_{image}$, $y_{min}* height_{image}$) and the lower right corner should be ($x_{max}*width_{image}$, $y_{max}* height_{image}$)). In the class column is an integer that corresponds to the classes in your label_map.pbtxt file. If you used the same classes in the annotation notebook, then class 1 is a worm and class 2 is an egg. Finally, the 'score' column is an indication of the confidence that the model has in this detection, with scores closer to 1 being highly confident and scores close to zero having very low confidence. 

If you wish to detect classes other than class 1 and 2 from the label_map.pbtxt file, modify the list of ```target_classes``` below to reflect the index-shifted classes (as described at the top of Step 6) you would like to be saved to the csv file. 





In [ ]:
import h5py
import pandas as pd
from inference_code import inferencing_tools
from object_detection.utils import label_map_util

save_path = './inferencing-results'
# Note: change this name so it corresponds to the h5 file you want to convert
h5_file = os.path.join(save_path, 'test_detections.h5')
target_classes = [0, 1]

xmins = []
xmaxs = []
ymins = []
ymaxs = []
classes = []
scores = []
frame_no = []


with h5py.File(h5_file, 'r') as hf:
  keys = list(hf.keys())
  print(keys)
  # look for the identifier at the end of the key - for movies this is the frame number, 
  # for images it is the image name
  frames = [txt.split('frame_')[-1] for txt in keys]
  for frame in frames:
    # get info about frame
    xmin, ymin, xmax, ymax, id, score = inferencing_tools.get_boxes_from_h5(frame, hf, target_classes)
    xmins.extend(xmin)
    ymins.extend(ymin)
    xmaxs.extend(xmax)
    ymaxs.extend(ymax)
    classes.extend(id)
    scores.extend(score)
    frame_no.extend([frame]*len(score))

data = {'frame': frame_no, 'xmin': xmins, 'xmax':xmaxs, 'ymin':ymins, 
        'ymax':ymaxs, 'class':classes, 'score':scores}
df = pd.DataFrame(data)

# save csv with detections to 'inferencing-results' directory
csv_filepath = './inferencing-results/bounding_boxes_from_folder.csv'
df.to_csv(csv_filepath, index=False)

### Step 7: Training more

---

If you're not satisifed with your model after training, it's straightforward to continue from where you left off. You can also add more data to your training set at this point. 
1. Prepare your Google Drive.
  *   Upload the new images into the 'images' directory in this Google Drive folder
  * If you are adding new classes, you will need to delete the file 'label_map.pbtxt' in the 'training' folder.
  *   Delete the train.record and test.record files in the 'annotations' folder. 

2.   Continue by working Steps 1-4 in this notebook again. 

3.   Before re-starting training, we need to edit the 'faster_rcnn.config' file in the 'training' folder. Go to line 104 of the file, which currently says 
```fine_tune_checkpoint: "COCO-trained-model/checkpoint/ckpt-0"```

  Assuming you want to start from the model you previously exported, you would change this line to 
```fine_tune_checkpoint: "exported-model/checkpoint/ckpt-0"```

4.   Delete all the files in the 'training' directory **except** the faster_rcnn.config file and the label_map.pbtxt file.

5.   Continue by working the remaining steps in this notebook.  